In [1]:
import os
# Install java"
#!sudo apt-get update
#!sudo apt-get install --fix-missing -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]

In [2]:
!java -version
!free -g

openjdk version "1.8.0_265"
OpenJDK Runtime Environment (build 1.8.0_265-8u265-b01-0+deb9u1-b01)
OpenJDK 64-Bit Server VM (build 25.265-b01, mixed mode)
              total        used        free      shared  buff/cache   available
Mem:             62           0          61           0           0          61
Swap:             0           0           0


In [3]:
#!pip install --ignore-installed -q pyspark==2.4.5
#!gsutil cp gs://hadoop-lib/gcs/gcs-connector-hadoop2-latest.jar /opt/conda/lib/python3.7/site-packages/pyspark/jars/
    
#!pip install --ignore-installed spark-nlp==2.5.1

In [4]:
!ps -ef | grep spark

jupyter   1823  1716  0 02:31 pts/0    00:00:00 /bin/sh -c ps -ef | grep spark
jupyter   1825  1823  0 02:31 pts/0    00:00:00 grep spark


In [5]:
import sparknlp
import json
import os
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from sparknlp.annotator import *
from sparknlp.base import *
from pyspark.sql.functions import from_unixtime
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.functions import *
from pyspark.sql.functions import explode, col
from pyspark.sql.functions import from_unixtime, to_date, asc, year, udf, explode, split, col, desc, length, rank, dense_rank, avg, sum
from pyspark.sql.window import Window
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.stat import Correlation
from pyspark.ml.clustering import BisectingKMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql.functions import col, to_timestamp,date_format
from pyspark import StorageLevel
import pyspark.sql.functions as F
from sparknlp.pretrained import PretrainedPipeline
from collections import Counter
from sparknlp.base import Finisher, DocumentAssembler
from sparknlp.annotator import (Tokenizer, Normalizer,LemmatizerModel, StopWordsCleaner)
from pyspark.ml import Pipeline
import matplotlib.pyplot as plt
from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *
from pyspark.ml.feature import Normalizer, SQLTransformer
from pyspark.ml.feature import BucketedRandomProjectionLSH
from pyspark.sql.functions import monotonically_increasing_id

import pandas as pd

def start():
    builder = SparkSession.builder \
        .appName("Spark NLP Licensed") \
        .master("local[2]") \
        .config("spark.driver.memory", "40G") \
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
        .config("spark.kryoserializer.buffer.max", "2000M") \
        .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.11:2.5.1") \
        .config("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
        .config("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
    return builder.getOrCreate()
spark = start()

In [6]:
spark.version
!ps -ef | grep spark

jupyter   1826  1716 45 02:31 ?        00:00:13 /usr/lib/jvm/java-8-openjdk-amd64/bin/java -cp /opt/conda/lib/python3.7/site-packages/pyspark/conf:/opt/conda/lib/python3.7/site-packages/pyspark/jars/* -Xmx40G org.apache.spark.deploy.SparkSubmit --conf spark.master=local[2] --conf spark.driver.memory=40G --conf spark.kryoserializer.buffer.max=2000M --conf spark.jars.packages=com.johnsnowlabs.nlp:spark-nlp_2.11:2.5.1 --conf spark.serializer=org.apache.spark.serializer.KryoSerializer --conf spark.app.name=Spark NLP Licensed --conf fs.AbstractFileSystem.gs.impl=com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS --conf fs.gs.impl=com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem pyspark-shell
jupyter   1988  1716  0 02:32 pts/0    00:00:00 /bin/sh -c ps -ef | grep spark
jupyter   1990  1988  0 02:32 pts/0    00:00:00 grep spark


In [7]:
primaryCorpus = spark.read.option("header","true").csv("file1.csv")
secondaryCorpus = spark.read.option("header","true").csv("file2.csv")

In [8]:
documentAssembler = DocumentAssembler().setInputCol("text").setOutputCol("document")

sentence = SentenceDetector()\
      .setInputCols("document")\
      .setOutputCol("sentence")\
      .setExplodeSentences(False)

tokenizer = Tokenizer()\
    .setInputCols(['sentence'])\
    .setOutputCol('token')

bertEmbeddings = BertEmbeddings\
 .pretrained('bert_base_cased', 'en') \
 .setInputCols(["sentence",'token'])\
 .setOutputCol("bert")\
 .setCaseSensitive(False)\
 .setPoolingLayer(0)

embeddingsSentence = SentenceEmbeddings() \
      .setInputCols(["sentence", "bert"]) \
      .setOutputCol("sentence_embeddings") \
      .setPoolingStrategy("AVERAGE")

embeddingsFinisher = EmbeddingsFinisher() \
    .setInputCols(["sentence_embeddings","bert"]) \
    .setOutputCols("sentence_embeddings_vectors", "bert_vectors") \
    .setOutputAsVector(True)\
    .setCleanAnnotations(False)


explodeVectors = SQLTransformer() \
.setStatement("SELECT EXPLODE(sentence_embeddings_vectors) AS features, * FROM __THIS__")

vectorNormalizer = Normalizer() \
    .setInputCol("features") \
    .setOutputCol("normFeatures") \
    .setP(1.0)

similartyChecker = BucketedRandomProjectionLSH(inputCol="features", outputCol="hashes", bucketLength=6.0,numHashTables=6)
    

bert_base_cased download started this may take some time.
Approximate size to download 389.2 MB
[OK!]


In [9]:
pipeline = Pipeline() \
      .setStages([documentAssembler,
        sentence,
        tokenizer,
        bertEmbeddings,
        embeddingsSentence,
        embeddingsFinisher,
        explodeVectors,
        vectorNormalizer,
        similartyChecker])

In [13]:
pipelineModel = pipeline.fit(primaryCorpus)
primaryDF = pipelineModel.transform(primaryCorpus)
secondaryDF = pipelineModel.transform(secondaryCorpus)

In [14]:
dfA = primaryDF.select("text","features","normFeatures").withColumn("lookupKey", md5("text")).withColumn("id",monotonically_increasing_id())
dfA.show()

+--------------------+--------------------+--------------------+--------------------+---+
|                text|            features|        normFeatures|           lookupKey| id|
+--------------------+--------------------+--------------------+--------------------+---+
|Wall Decals Lamp ...|[-0.0029230886138...|[-1.6475124912950...|bbc5a89d7cf3354ea...|  0|
|iphone charger ph...|[0.41062018275260...|[0.00185856293451...|37c2b6ab956f9ebd6...|  1|
+--------------------+--------------------+--------------------+--------------------+---+



In [15]:
dfB = secondaryDF.select("text","features","normFeatures").withColumn("id",monotonically_increasing_id())
dfB.show()

+--------------------+--------------------+--------------------+---+
|                text|            features|        normFeatures| id|
+--------------------+--------------------+--------------------+---+
|Curtains & Valanc...|[-0.1797953695058...|[-0.0012058615261...|  0|
|iphone case Apple...|[-0.2585198283195...|[-8.8132988264745...|  1|
+--------------------+--------------------+--------------------+---+



In [22]:
#print("Approximately joining dfA and dfB :")
pipelineModel.stages[8].approxSimilarityJoin(dfA, dfB, 100, distCol="distance")\
     .where(col("datasetA.id") == col("datasetB.id")) \
     .select(col("datasetA.text").alias("idA"), \
            col("datasetB.text").alias("idB"), \
            col("distance")).show()


+--------------------+--------------------+-----------------+
|                 idA|                 idB|         distance|
+--------------------+--------------------+-----------------+
|iphone charger ph...|iphone case Apple...|9.144942521650227|
|Wall Decals Lamp ...|Curtains & Valanc...|4.948642541753079|
+--------------------+--------------------+-----------------+



## Approach 2

In [ ]:
from pyspark.sql.functions import PandasUDFType, pandas_udf
import pyspark.sql.functions as F

dfA = dfA.withColumnRenamed('text','primaryText').withColumnRenamed('features', 'primaryFeatures')

dfB = dfB.withColumnRenamed('text','secondaryText').withColumnRenamed('features', 'secondaryFeatures')

joinedDF = dfA.join(dfB, "id", "inner").drop("id","normFeatures")

joinedDF.show()

In [ ]:
from scipy.spatial.distance import cosine

finalDF = joinedDF.toPandas()

finalDF['cosine'] = finalDF.apply(lambda row: 1-cosine(row['primaryFeatures'], row['secondaryFeatures']), axis=1)
finalDF